## Previsão de Inadimplência

### Leitura dos Dados

In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [2]:
train_df = pd.read_parquet('data/train.parquet')
test_df = pd.read_parquet('data/test.parquet')
sample = pd.read_parquet('data/sub.parquet')

### Feature Engineering

Após seleção inicial de features realizamos a ordenação por data e normalização das colunas `divida_total`, `transacionado` e `pagamento_diario` por meio da divisão pelo `valor_emprestado`. Desta forma é possível comparar empresas cujo empréstimo e receitas tenham grandes diferenças. Em seguida extraímos os valores destas colunas para os primeiros 90 dias, preenchendo quando necessário. Este preenchimento foi realizado levando em conta um cenário pessimista, em que a dívida se mantém e não são realizados novos pagamentos.

In [3]:
def sort(df):
    df = df.sort_values(by=['id', 'dias_pos_desembolso'])

def normalize(df):
    df['divida_total'] /= df['valor_emprestado']
    df['transacionado'] /= df['valor_emprestado']
    df['pagamento_diario'] /= df['valor_emprestado']

def padding(x, n, mode):
    x = np.array(x)
    if x.size > n:
        return x[:n]
    return np.pad(x, (0, n - x.size), mode=mode)

def format_features(df, n=90):
    ids = df.id.unique()
    x = np.zeros((len(ids), 3*n))
    for i, idx in enumerate(ids):
        dfi = df.query(f'id == {idx}')
        debt = dfi.divida_total.values
        debt = padding(debt, n, 'edge')
        pay = dfi.pagamento_diario.values
        pay = padding(pay, n, 'constant')
        sales = dfi.transacionado.values
        sales = padding(sales, n, 'constant')
        x[i, :] = np.hstack((debt, pay, sales))
    return x

def format_targets(df):
    targets = df.groupby('id').first()
    targets = targets.y.values
    return targets

In [4]:
sort(train_df)
normalize(train_df)

In [5]:
x = format_features(train_df)
y = format_targets(train_df)
x_train, x_valid, y_train, y_valid = train_test_split(x, y, train_size=.8)

### Modelagem

Escolhemos os modelos de regressão logística e máquina de vetores suporte com classes balanceadas após uma bateria de iterações.

In [6]:
def assess_model(estimator, x, y):
    y_pred = estimator.predict(x)
    y_proba = estimator.predict_proba(x)[:, 1]
    print(classification_report(y, y_pred, digits=4), end='')
    print(f'\tauroc\t{roc_auc_score(y, y_proba):.4f}\n')

#### Modelo Base

In [7]:
model = LogisticRegression(class_weight='balanced', max_iter=1000)
model = model.fit(x_train, y_train)

assess_model(model, x_train, y_train)
assess_model(model, x_valid, y_valid)

              precision    recall  f1-score   support

         0.0     0.9914    0.8050    0.8885     13264
         1.0     0.1668    0.8478    0.2788       611

    accuracy                         0.8068     13875
   macro avg     0.5791    0.8264    0.5836     13875
weighted avg     0.9551    0.8068    0.8616     13875
	auroc	0.9066

              precision    recall  f1-score   support

         0.0     0.9894    0.7920    0.8798      3312
         1.0     0.1577    0.8217    0.2646       157

    accuracy                         0.7933      3469
   macro avg     0.5736    0.8068    0.5722      3469
weighted avg     0.9518    0.7933    0.8519      3469
	auroc	0.8850



#### Modelo Final

In [8]:
model = SVC(class_weight='balanced', probability=True)
model.fit(x_train, y_train)

assess_model(model, x_train, y_train)
assess_model(model, x_valid, y_valid)

              precision    recall  f1-score   support

         0.0     0.9915    0.7823    0.8746     13264
         1.0     0.1531    0.8543    0.2597       611

    accuracy                         0.7855     13875
   macro avg     0.5723    0.8183    0.5671     13875
weighted avg     0.9546    0.7855    0.8475     13875
	auroc	0.9027

              precision    recall  f1-score   support

         0.0     0.9914    0.7699    0.8668      3312
         1.0     0.1505    0.8599    0.2562       157

    accuracy                         0.7740      3469
   macro avg     0.5710    0.8149    0.5615      3469
weighted avg     0.9534    0.7740    0.8391      3469
	auroc	0.8906



### Discussão

No futuro gostaria de melhorar a modelagem explorando a aplicação de modelos de séries temporais, principalmente eliminando a restrição aos primeiros 90 dias. Talvez flexibilizando para atualizar a probabilidade semanalmente. Em geral não tive ideia boa o suficiente, que fizesse uma diferença satisfatória. Nem as tentativas de otimização de modelos mais complexos deram certo. *C'est la vie*.

### Entrega

In [9]:
sort(test_df)
normalize(test_df)

In [10]:
model = model.fit(x, y)

In [11]:
x_test = format_features(test_df)
y_test_pred = model.predict(x_test)
y_test_prob = model.predict_proba(x_test)

In [12]:
sub_df = pd.DataFrame({
    'id': test_df.id.unique(),
    'y_pred': y_test_pred,
    'y_prob': y_test_prob[:, 1],
})
sub_df.to_parquet('solution.parquet')